In [3]:
import math
import numpy as np
import cv2

def gaussianSmoothing(image):
    """
    Applies 7x7 Gaussian Filter to the image using convolution.
    """
    gaussian_kernel = (1.0 / 140.0) * np.array([
        [1, 1, 2, 2, 2, 1, 1],
        [1, 2, 2, 4, 2, 2, 1],
        [2, 2, 4, 8, 4, 2, 2],
        [2, 4, 8, 16, 8, 4, 2],
        [2, 2, 4, 8, 4, 2, 2],
        [1, 2, 2, 4, 2, 2, 1],
        [1, 1, 2, 2, 2, 1, 1]
    ])
    # Using OpenCV's filter2D for convolution with padding to maintain size
    smoothed = cv2.filter2D(image, -1, gaussian_kernel, borderType=cv2.BORDER_REPLICATE)
    return smoothed

def computeGradients(image):
    """
    Computes horizontal (Gx) and vertical (Gy) gradients using Prewitt operators.
    """
    prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float32)
    prewitt_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=np.float32)

    Gx = cv2.filter2D(image, cv2.CV_32F, prewitt_x, borderType=cv2.BORDER_REPLICATE)
    Gy = cv2.filter2D(image, cv2.CV_32F, prewitt_y, borderType=cv2.BORDER_REPLICATE)

    return Gx, Gy

def nonMaxSuppression(mag, angle):
    """
    Applies non-maxima suppression to the gradient magnitude.
    """
    height, width = mag.shape
    suppressed = np.zeros_like(mag)

    for i in range(1, height-1):
        for j in range(1, width-1):
            theta = angle[i, j]
            # Convert angle to 0-180 range
            theta = theta % 180

            # Determine neighboring pixels based on gradient direction
            if (0 <= theta < 22.5) or (157.5 <= theta <= 180):
                neighbors = [mag[i, j-1], mag[i, j+1]]
            elif 22.5 <= theta < 67.5:
                neighbors = [mag[i-1, j+1], mag[i+1, j-1]]
            elif 67.5 <= theta < 112.5:
                neighbors = [mag[i-1, j], mag[i+1, j]]
            else: # 112.5 <= theta < 157.5
                neighbors = [mag[i-1, j-1], mag[i+1, j+1]]

            # Suppress non-maxima
            if mag[i, j] >= max(neighbors):
                suppressed[i, j] = mag[i, j]
            else:
                suppressed[i, j] = 0

    return suppressed

def pTileThreshold(suppressed, percent):
    """
    Applies p-tile thresholding to the suppressed image.
    """
    # Flatten and sort the non-zero magnitudes
    non_zero = suppressed[suppressed > 0]
    sorted_mag = np.sort(non_zero)

    # Calculate threshold index
    threshold_idx = max(0, len(sorted_mag) - 1 - int(percent/100 * len(sorted_mag)))
    threshold = sorted_mag[threshold_idx]

    # Apply threshold
    edges = np.zeros_like(suppressed, dtype=np.uint8)
    edges[suppressed >= threshold] = 255
    return edges

if __name__ == "__main__":
    # Read image
    image = cv2.imread('/content/dataa/4.tif', 0)

    # Step 1: Gaussian Smoothing
    smoothed = gaussianSmoothing(image)
    cv2.imwrite('/content/filter_gauss.jpg', smoothed)

    # Step 2: Compute Gradients
    Gx, Gy = computeGradients(smoothed)

    # Step 3: Compute Magnitude and Angle
    mag = np.sqrt(Gx**2 + Gy**2) / (3 * np.sqrt(2))  # Normalize
    angle = np.rad2deg(np.arctan2(Gy, Gx)) % 180

    # Step 4: Non-Maxima Suppression
    suppressed = nonMaxSuppression(mag, angle)
    cv2.imwrite('/content/suppressed.jpg', suppressed)

    # Step 5: Thresholding with p-tile
    edges_10 = pTileThreshold(suppressed, 10)
    edges_30 = pTileThreshold(suppressed, 30)
    edges_50 = pTileThreshold(suppressed, 50)

    cv2.imwrite('/content/edges_10.jpg', edges_10)
    cv2.imwrite('/content/edges_30.jpg', edges_30)
    cv2.imwrite('/content/edges_50.jpg', edges_50)

In [6]:
import math
import numpy as np
import cv2

def gaussianSmoothing(image):
    """
    Applies 7x7 Gaussian Filter to the image using convolution.
    """
    gaussian_kernel = (1.0 / 140.0) * np.array([
        [1, 1, 2, 2, 2, 1, 1],
        [1, 2, 2, 4, 2, 2, 1],
        [2, 2, 4, 8, 4, 2, 2],
        [2, 4, 8, 16, 8, 4, 2],
        [2, 2, 4, 8, 4, 2, 2],
        [1, 2, 2, 4, 2, 2, 1],
        [1, 1, 2, 2, 2, 1, 1]
    ])
    # Using OpenCV's filter2D for convolution with padding to maintain size
    smoothed = cv2.filter2D(image, -1, gaussian_kernel, borderType=cv2.BORDER_REPLICATE)
    return smoothed

def computeGradients(image):
    """
    Computes horizontal (Gx) and vertical (Gy) gradients using Prewitt operators.
    """
    prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float32)
    prewitt_y = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=np.float32)

    Gx = cv2.filter2D(image, cv2.CV_32F, prewitt_x, borderType=cv2.BORDER_REPLICATE)
    Gy = cv2.filter2D(image, cv2.CV_32F, prewitt_y, borderType=cv2.BORDER_REPLICATE)

    return Gx, Gy

def nonMaxSuppression(mag, angle):
    """
    Applies non-maxima suppression to the gradient magnitude.
    """
    height, width = mag.shape
    suppressed = np.zeros_like(mag)

    for i in range(1, height-1):
        for j in range(1, width-1):
            theta = angle[i, j]
            # Convert angle to 0-180 range
            theta = theta % 180

            # Determine neighboring pixels based on gradient direction
            if (0 <= theta < 22.5) or (157.5 <= theta <= 180):
                neighbors = [mag[i, j-1], mag[i, j+1]]
            elif 22.5 <= theta < 67.5:
                neighbors = [mag[i-1, j+1], mag[i+1, j-1]]
            elif 67.5 <= theta < 112.5:
                neighbors = [mag[i-1, j], mag[i+1, j]]
            else: # 112.5 <= theta < 157.5
                neighbors = [mag[i-1, j-1], mag[i+1, j+1]]

            # Suppress non-maxima
            if mag[i, j] >= max(neighbors):
                suppressed[i, j] = mag[i, j]
            else:
                suppressed[i, j] = 0

    return suppressed

def pTileThreshold(suppressed, percent):
    """
    Applies p-tile thresholding to the suppressed image.
    """
    # Flatten and sort the non-zero magnitudes
    non_zero = suppressed[suppressed > 0]
    sorted_mag = np.sort(non_zero)

    # Calculate threshold index
    threshold_idx = max(0, len(sorted_mag) - 1 - int(percent/100 * len(sorted_mag)))
    threshold = sorted_mag[threshold_idx]

    # Apply threshold
    edges = np.zeros_like(suppressed, dtype=np.uint8)
    edges[suppressed >= threshold] = 255
    return edges

if __name__ == "__main__":
    # Read image
    image = cv2.imread('/content/dataa/5.tif', 0)

    # Step 1: Gaussian Smoothing
    smoothed = gaussianSmoothing(image)
    cv2.imwrite('/content/filter_gauss.jpg', smoothed)

    # Step 2: Compute Gradients
    Gx, Gy = computeGradients(smoothed)

    # Step 3: Compute Magnitude and Angle
    mag = np.sqrt(Gx**2 + Gy**2) / (3 * np.sqrt(2))  # Normalize
    angle = np.rad2deg(np.arctan2(Gy, Gx)) % 180

    # Step 4: Non-Maxima Suppression
    suppressed = nonMaxSuppression(mag, angle)
    cv2.imwrite('/content/suppressed.jpg', suppressed)

    # Step 5: Thresholding with p-tile
    edges_10 = pTileThreshold(suppressed, 10)
    edges_30 = pTileThreshold(suppressed, 30)
    edges_50 = pTileThreshold(suppressed, 50)

    cv2.imwrite('/content/edges_10.jpg', edges_10)
    cv2.imwrite('/content/edges_30.jpg', edges_30)
    cv2.imwrite('/content/edges_50.jpg', edges_50)